In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [3]:
yelp = pd.read_csv('yelp.csv')

In [4]:
yelp = yelp[['text','stars']]

In [5]:
yelp_class = yelp[(yelp['stars']==1)|(yelp['stars']==5)]

In [6]:
yelp_class.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5
6,Drop what you're doing and drive here. After I...,5


In [7]:
yelp_class['text'] = yelp['text'].apply(lambda x: x.lower())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
yelp_class['text'] = yelp['text'].apply(lambda x:re.sub('[^a-zA-z0-9\s]','',x))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
print(yelp_class[ yelp_class['stars'] == 1].size)
print(yelp_class[ yelp_class['stars'] == 5].size)

1498
6674


In [10]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(yelp_class['text'].values)
X = tokenizer.texts_to_sequences(yelp_class['text'].values)
X = pad_sequences(X)

In [11]:
embed_dim = 128
lstm_out = 196

model = Sequential()

model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))

model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 789, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 789, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
Y = pd.get_dummies(yelp_class['stars']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2737, 789) (2737, 2)
(1349, 789) (1349, 2)


In [13]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
 - 330s - loss: 0.4633 - accuracy: 0.8184
Epoch 2/7
 - 469s - loss: 0.2945 - accuracy: 0.8853
Epoch 3/7
 - 464s - loss: 0.1753 - accuracy: 0.9364
Epoch 4/7
 - 470s - loss: 0.1225 - accuracy: 0.9536
Epoch 5/7
 - 468s - loss: 0.1074 - accuracy: 0.9602
Epoch 6/7
 - 458s - loss: 0.0719 - accuracy: 0.9748
Epoch 7/7
 - 386s - loss: 0.0584 - accuracy: 0.9795


In [17]:
text = ['I have no idea why some people give bad']
text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=789, dtype='int32', value=0)
# print(twt)
sentiment = model.predict(text,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

positive
